# PSYC0021: Tutorial to Extract Features from PPG Signal

## Import packages

In [ ]:
%matplotlib inline

import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import heartpy as hp
import pandas as pd
pd.set_option("display.precision", 2)
from scipy.interpolate import interp1d
from utils.process_ppg import load_PPG_signal, get_filtered_ppg, get_ppg_measures_batch

## Specify directory paths and the file containing PPG signal

In [ ]:
basepath = 'data/' 
# filepath = os.path.join(basepath, r'RawPPG/s001/ppgSignal_low_2021-03-04 221859.334287.csv') #Change this to analyze PPG singal of your interest
# filepath = os.path.join(basepath, r'RawPPG/s001/ppgSignal_moderate_2021-03-17 23_47_06.716435.csv') #Change this to analyze PPG singal of your interest
filepath = os.path.join(basepath, r'RawPPG/s001/ppgSignal_high_2021-03-17 23_49_47.641078.csv') #Change this to analyze PPG singal of your interest
sample_rate = 30.0  #Change this as per the frame-rate of your smart-phone. If you used PPG.apk to acquire the data, then you shall keep this unchanged

## Load the signal

In [ ]:
raw_signal, tElapsed = load_PPG_signal(filepath)
exertion_level = os.path.basename(filepath).split('_')[1]

## Visualize raw signal

In [ ]:
x_axis = [i/sample_rate for i in range(len(raw_signal))]
plt.plot(x_axis, raw_signal)
plt.xlabel('Time(seconds)')
plt.title('Raw PPG Signal: ' + 'Exertion - ' + exertion_level)

## Filter the signal and Plot


In [ ]:
filtered = get_filtered_ppg(raw_signal, sample_rate=sample_rate)
if len(filtered) > 30*sample_rate:
    start_indx = int(((len(filtered)//2) - (15*sample_rate)))
    end_indx = int(((len(filtered)//2) + (15*sample_rate)))
    filtered = filtered[start_indx : end_indx]
plt.figure(figsize=(12,4))
x_axis = [i/sample_rate for i in range(len(filtered))]
plt.plot(x_axis, filtered)
plt.xlabel('Time(seconds)')
plt.title('Filtered PPG Signal: ' + 'Exertion - ' + exertion_level)

## Compute the features using 'heartpy' Python package <br>
Plot the signal with peaks identified and print the measures derived

In [ ]:
wd, m = hp.process(filtered, sample_rate = sample_rate)
# print(wd.keys())
m['exertion_level'] = exertion_level

#display computer features
for measure in m.keys():
    print(measure, m[measure])

### Plot PPG signal with detected and rejected peaks

In [ ]:
#Plot PPG Signal with Peaks
plt.figure(figsize=(12,4))
x_axis = [i/sample_rate for i in range(len(wd['hr']))]
plt.plot(x_axis, wd['hr'])
plt.plot(np.array(wd['peaklist'])/sample_rate, wd['hr'][wd['peaklist']], '.', color='g', label='Peaks')
plt.plot(np.array(wd['removed_beats'])/sample_rate, wd['hr'][wd['removed_beats']], '.', color='r', label='Rejected Peaks')
mx = max(wd['hr'])
plt.text(2,  mx, 'HR: ' + str(int(round(m['bpm']))))
plt.text(5, mx, 'IBI: ' +  str(int(round(m['ibi']))))
plt.text(8, mx, 'SDNN: ' +  str(int(round(m['sdnn']))))
plt.text(11, mx, 'Resp: ' +  str(int(round(m['breathingrate'] * 60))))

plt.legend()
plt.xlabel('Time(seconds)')
plt.title('Filtered PPG Signal with Detected Peaks: ' + 'Exertion - ' + exertion_level)

### Plot Breathing signal derived from PPG signal

In [ ]:
# Plot breathing signal derived from PPG signal
x_axis_br = [i/1000.0 for i in range(len(wd['breathing_signal']))]
plt.plot(x_axis_br, wd['breathing_signal'])
plt.xlabel('Time(seconds)')
plt.title('Breathing Signal')

## Batch processing for feature extraction

In [ ]:
# datapath = os.path.join(basepath, 'RawPPG')
get_ppg_measures_batch(basepath, outdir=basepath, sample_rate=sample_rate)

## Next Steps
As an outcome of this tutorial, two files will be generated (in basepath directory) comprising of PPG derived features for each of the signals present in the directory <br>
1. PPG_features.csv -> To be used with WEKA for further ML Model Development <br>
2. PPG_features.pkl -> To be used for ML Model Development using Python